In [69]:
import pandas as pd
import numpy as np

def strip_nans(x):
    # gets rid of nans in a list
    if isinstance(x, list):
        x_strip = ""
        for a in x:
            if a == a:
                # not nan
                x_strip = x_strip + a.strip()
        return x_strip
    else:
        return x.strip()

# Closure:
# emergency declaration = e/E; restaurant closure = r/R
# border screening = b/B; travel restrictions= t/T; border closures = c/C
# shelter-in-place = l/L; gathering limitations= g/G; k-12 school closures = s/S
# non-essential business closure = n/N
# face covering requirement = f/F

# Opening:
# emergency declaration lifted = a/A; 
# travel restrictions lifted= d/D;
# border closures lifted = k/K;
# shelter-in-place lifted = h/H;
# night-time curfew lifted= i/I; 
# Banning gatherings lifted = j/J;
# Face covering requirement lifted =p/P;
# k-12 school open = m/M;
# Bar and Dine-in Restaurant open = q/Q;
# Non-essential Businesses open = o/O

In [115]:
def append_most_recent_events (df, x): 
    df = df[df['Country_Region'] == x['Country_Region']][df['lockdown_date'] < x['lockdown_date']]
    if (x['coverage'] == 'General'):
        df = df[df['coverage'] == x['coverage']]
    df = df.sort_values(by = ['lockdown_date'])
    for i in {'Travel Restrictions', 'Gathering Limitations', 'Shelter-in-place Order', 'K-12 School Closure', 'Non-essential Businesses Closure'}:
        s = ""
        for index, r in df.iterrows():
            if (r[i] != ''):
                s = r[i]
        x[i] = x[i] + "*"  + s
    return x

def interpret_events (y, regional_flag, not_the_first_flag, s, emo_s, lockdown_type):
    a = y[lockdown_type].lower().split('*')
    if (lockdown_type == 'Shelter-in-place Order'): 
        readable_closed = " Shelter-in-place Order"; readable_open = " Shelter-in-place Order Lifted";
        emoji_close = "L"; emoji_open = "H"
    if (lockdown_type == 'Gathering Limitations'): 
        readable_closed = " Gatherings Banned"; readable_open = " Gathering Ban Lifted";
        emoji_close = "G"; emoji_open = "J"
    if (lockdown_type == 'K-12 School Closure'): 
        readable_closed = " Schools Closed"; readable_open = " Schools Reopened";
        emoji_close = "S"; emoji_open = "M"
    if (lockdown_type == 'Non-essential Businesses Closure'): 
        readable_closed = " Businesses Closed"; readable_open = " Businesses Reopened";
        emoji_close = "N"; emoji_open = "O"
    if ((a[0] != "") and (("require" in a[0]) or ("require" in a[1])) and 
        (not (("require" in a[0]) and ("require" in a[1])))):
        if not_the_first_flag == 1:
            s = s + ","
        if ("require" in a[0]):
            s = s + readable_closed
            emo_s = (emo_s + emoji_close.lower()) if (regional_flag == 1) else (emo_s + emoji_close)
        else:
            s = s + readable_open
            emo_s = (emo_s + emoji_open.lower()) if (regional_flag == 1) else (emo_s + emoji_open)
        not_the_first_flag = 1
    
    return not_the_first_flag, s, emo_s 

def create_lockdown_type_world_new_export(df, x, emo_flag):
    s = r = emo_s = ""
    x = append_most_recent_events (df, x)
    regional_flag = closure_flag = not_the_first_flag = 0
    if x['coverage'] == 'Targeted':
        r = 'Regional'
        regional_flag = 1
    for i in {'Shelter-in-place Order', 'Gathering Limitations','K-12 School Closure', 'Non-essential Businesses Closure'}:
        not_the_first_flag, s, emo_s = interpret_events (x, regional_flag, not_the_first_flag, s, emo_s, i)
    if emo_flag == 1:
        return emo_s
    if s == "":
        return s  # Ensures just "Regional" is not returned
    return (r + s).strip()


quarantine_csv = '../data/quarantine-activity-world-new-export.csv'
quarantine_df = pd.read_csv(quarantine_csv)
quarantine_df = quarantine_df.rename(
    columns={'date': 'lockdown_date', 'Date Enacted': 'lockdown_date',
                      'country_name': 'Country_Region', 'Coverage': 'coverage'}
)

quarantine_df = quarantine_df.groupby(['Country_Region', 'lockdown_date', 'coverage']).agg({
    'Travel Restrictions': list,
    'Gathering Limitations': list,
    'Shelter-in-place Order': list,
    'K-12 School Closure': list,
    'Non-essential Businesses Closure': list
}).reset_index()
quarantine_df = quarantine_df.applymap(strip_nans)
if quarantine_csv.endswith('quarantine-activity-world-new-export.csv'):
    lockdown_mapper = create_lockdown_type_world_new_export
else:
    lockdown_mapper = create_lockdown_type_world

    
quarantine_df['lockdown_type'] = quarantine_df.apply(lambda x: lockdown_mapper(quarantine_df, x, 0), axis=1)
quarantine_df['emoji_string'] = quarantine_df.apply(lambda x: lockdown_mapper(quarantine_df, x, 1), axis=1)
quarantine_df['lockdown_type'].replace('', np.nan, inplace=True)
quarantine_df = quarantine_df.dropna(subset=['lockdown_type'])
quarantine_df = quarantine_df.groupby(['lockdown_date', 'Country_Region']).agg({
    'lockdown_type': lambda col: '; '.join(col),
    'emoji_string': lambda col: ''.join(col)
}).reset_index()
    
quarantine_df

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,lockdown_date,Country_Region,lockdown_type,emoji_string
0,01-21-2020,Mongolia,Gatherings Banned,G
1,01-23-2020,China,"Regional Gatherings Banned, Shelter-in-place O...",gl
2,01-23-2020,Macao,Gatherings Banned,G
3,01-25-2020,Hong Kong,Schools Closed,S
4,01-26-2020,China,"Schools Closed, Businesses Closed",SN
...,...,...,...,...
496,04-27-2020,Nigeria,"Shelter-in-place Order, Businesses Closed",LN
497,04-27-2020,Switzerland,Shelter-in-place Order Lifted,H
498,04-30-2020,Norway,Businesses Closed,N
499,05-03-2020,San Marino,"Shelter-in-place Order Lifted, Businesses Closed",HN
